<a href="https://colab.research.google.com/github/ValaryLim/Spam-Ham-Email-Classifier/blob/master/Spam_Ham_Email_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Spam-Ham Email Classifier </h1>

<h2> Importing Packages & Dataset </h2>
Dataset was obtained from: https://www.kaggle.com/karthickveerakumar/spam-filter/downloads/spam-filter.zip/1

In [0]:
# Connect google drive
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
! pip install glove_python

     |████████████████████████████████| 266kB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
import os
import re
import sys
import numpy as np
import pandas as pd

# embedding
import glove

# train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant



In [0]:
df = pd.read_csv('/content/drive/My Drive/Personal Projects/Deep Learning Jump-Start Workshop/emails.csv')

In [0]:
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


<h2> Embedding </h2>

<h3> Loading a pre-train word embedding </h3>
Loading a pre-trained word embedding from http://nlp.stanford.edu/projects/glove/, created from a 6 billion tokens corpus, with results as 50d vectors.

In [0]:
!wget -qq https://www.dropbox.com/s/v14xhvjmfniraf3/glove6b100dtxt.zip
# !wget -qq http://nlp.stanford.edu/data/glove.6B.zip
  
!unzip glove6b100dtxt.zip

Archive:  glove6b100dtxt.zip
replace glove.6B.100d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

<h3> Build index mapping words in the embedding set to their embedding vector </h3>

In [0]:
embeddings_index = {}

with open(os.path.join('', 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Get length of word vectors
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
# Example glove embedding
embeddings_index['spam']

array([-0.32896 ,  0.4371  , -0.1109  , -1.3661  , -0.070062,  1.0553  ,
        0.11511 , -1.5601  ,  0.52192 ,  0.13506 ,  0.39288 , -0.4516  ,
       -0.67174 , -0.33007 , -0.28505 ,  0.2817  ,  0.10494 , -0.14796 ,
        0.49491 , -0.10761 , -0.2568  ,  0.049184,  0.80427 ,  0.53083 ,
        0.04467 ,  0.83403 ,  0.30387 ,  0.84311 , -0.36066 , -1.1358  ,
        0.61644 ,  0.85332 ,  0.74056 ,  0.32785 ,  0.36279 , -0.055806,
       -0.63322 , -0.6299  ,  0.21147 ,  0.060588,  0.26387 , -0.022226,
       -0.95656 , -0.71311 ,  0.1739  ,  0.11947 , -0.7337  , -0.49539 ,
        0.17028 , -0.1535  ,  0.87158 ,  1.0749  , -0.33695 , -0.28382 ,
       -1.6008  , -0.092306,  0.38466 , -0.29229 ,  0.97992 ,  0.063228,
       -0.44015 , -0.080327, -0.2629  , -0.51111 ,  0.15873 ,  0.80866 ,
        0.31916 ,  0.15911 ,  1.1527  , -0.3635  , -0.22589 , -0.83732 ,
        0.57972 ,  0.35771 , -0.22302 , -0.026206, -0.55764 , -0.37955 ,
       -0.55529 ,  0.2698  ,  0.75772 , -1.0138  , 

<h2> Processing Text Dataset </h2>

<h3> Clean the data </h3>

In [0]:
# Viewing the data
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [0]:
# Change all text to lowercase
df['text'] = [entry.lower() for entry in df['text']]

In [0]:
# Remove numbers
df['text'] = [''.join([i for i in s if not i.isdigit()]) for s in df['text']]

In [0]:
# Remove punctuation
df['text'] = [re.sub(r'[^\w\s]','', entry) for entry in df['text']]

In [0]:
# Remove extra spaces
df['text'] = [re.sub(' +', ' ', entry) for entry in df['text']]

In [0]:
df.head()

,text,spam
0,subject naturally irresistible your corporate ...,1
1,subject the stock trading gunslinger fanny is ...,1
2,subject unbelievable new homes made easy im wa...,1
3,subject color printing special request additio...,1
4,subject do not have money get software cds fro...,1


In [0]:
texts = df['text']
texts.head()

0    subject naturally irresistible your corporate ...
1    subject the stock trading gunslinger fanny is ...
2    subject unbelievable new homes made easy im wa...
3    subject color printing special request additio...
4    subject do not have money get software cds fro...
Name: text, dtype: object

<h3> Vectorize the words to run through model </h3>

In [0]:
# Limiting the number of words to 20000 so model trains faster
MAX_NUM_WORDS = 20000

# Vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 33741 unique tokens.


In [0]:
# Viewing the word index
word_index

{'the': 1,
 'to': 2,
 'and': 3,
 'of': 4,
 'a': 5,
 'you': 6,
 'in': 7,
 'i': 8,
 'for': 9,
 'enron': 10,
 'on': 11,
 'is': 12,
 'ect': 13,
 'subject': 14,
 'this': 15,
 'that': 16,
 'your': 17,
 'be': 18,
 'with': 19,
 'we': 20,
 'vince': 21,
 'will': 22,
 'have': 23,
 'at': 24,
 'from': 25,
 'it': 26,
 'are': 27,
 's': 28,
 'as': 29,
 'hou': 30,
 'by': 31,
 'com': 32,
 'or': 33,
 'if': 34,
 'am': 35,
 'please': 36,
 'kaminski': 37,
 'not': 38,
 'would': 39,
 'me': 40,
 'our': 41,
 'can': 42,
 're': 43,
 'cc': 44,
 'j': 45,
 'my': 46,
 'an': 47,
 'pm': 48,
 'all': 49,
 'any': 50,
 'time': 51,
 'research': 52,
 'has': 53,
 'do': 54,
 'thanks': 55,
 'know': 56,
 'may': 57,
 'about': 58,
 'group': 59,
 'information': 60,
 'was': 61,
 'new': 62,
 'he': 63,
 'e': 64,
 'energy': 65,
 'like': 66,
 'more': 67,
 'risk': 68,
 'but': 69,
 'one': 70,
 'business': 71,
 'get': 72,
 'power': 73,
 'email': 74,
 'let': 75,
 'up': 76,
 'us': 77,
 'so': 78,
 'also': 79,
 'what': 80,
 'message': 81,
 'ma

<h3> Pad sequence to keep constant length </h3>

In [0]:
# Set maximum sequence length to 500 to speed up training
MAX_SEQUENCE_LENGTH = 500

data = pad_sequences(sequences, maxlen = MAX_SEQUENCE_LENGTH)
data

array([[   0,    0,    0, ...,  640,   38,  187],
       [   0,    0,    0, ..., 3784, 7573,   98],
       [   0,    0,    0, ...,  630,   25,    6],
       ...,
       [   0,    0,    0, ..., 1720,  261,  411],
       [   0,    0,    0, ...,  252,   21,   37],
       [   0,    0,    0, ..., 3689,  470,  251]], dtype=int32)

<h3> Split data into train and test sets </h3>

In [0]:
x_train, x_test, y_train, y_test = train_test_split(data, df['spam'], test_size= 0.3, random_state=42)

In [0]:
x_train

array([[    0,     0,     0, ...,  3015,    12,    33],
       [    2,   219,   304, ...,  1354,     2,   403],
       [    0,     0,     0, ...,  1237,    55,    92],
       ...,
       [    0,     0,     0, ...,     1,  2188,    21],
       [    0,     0,     0, ..., 10670,  2869,   251],
       [    0,     0,     0, ...,     6,    38,   187]], dtype=int32)

In [0]:
# One hot encode y_train and y_test data
y_train_1h = to_categorical(y_train)
y_test_1h = to_categorical(y_test)

In [0]:
y_train_1h

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]], dtype=float32)

<h2> Modelling </h2>
<h3> Create embedding layer </h3>

In [0]:
EMBEDDING_DIM = 100

# Prepare embedding matrix for embedding layer
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

<h3> Define the model </h3>

In [0]:
# Number of classes: spam and ham (2)
NUM_CLASSES = 2

# Preparing the layers
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = LSTM(128)(embedded_sequences)
preds = Dense(NUM_CLASSES, activation='softmax')(x)

In [0]:
# Building the model
model = Model(sequence_input, preds)

In [0]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 500, 100)          2000100   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 258       
Total params: 2,117,606
Trainable params: 117,506
Non-trainable params: 2,000,100
_________________________________________________________________


<h3> Compile the model </h3>

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [0]:
model.fit(x_train, y_train_1h,
          batch_size=128,
          epochs=10,
          validation_data=(x_test, y_test_1h))

Train on 4009 samples, validate on 1719 samples
Epoch 1/10
4009/4009 [==============================] - 19s 5ms/sample - loss: 0.3920 - acc: 0.8204 - val_loss: 0.3005 - val_acc: 0.8999
Epoch 2/10
4009/4009 [==============================] - 18s 5ms/sample - loss: 0.2628 - acc: 0.8955 - val_loss: 0.2306 - val_acc: 0.9075
Epoch 3/10
4009/4009 [==============================] - 18s 5ms/sample - loss: 0.1936 - acc: 0.9239 - val_loss: 0.1948 - val_acc: 0.9267
Epoch 4/10
4009/4009 [==============================] - 18s 5ms/sample - loss: 0.1628 - acc: 0.9356 - val_loss: 0.1212 - val_acc: 0.9558
Epoch 5/10
4009/4009 [==============================] - 18s 5ms/sample - loss: 0.1399 - acc: 0.9444 - val_loss: 0.1238 - val_acc: 0.9546
Epoch 6/10
4009/4009 [==============================] - 18s 5ms/sample - loss: 0.1083 - acc: 0.9603 - val_loss: 0.1228 - val_acc: 0.9540
Epoch 7/10
4009/4009 [==============================] - 18s 5ms/sample - loss: 0.0979 - acc: 0.9638 - val_loss: 0.1098 - val_acc: 

<h3> Model Evaluation </h3>

In [0]:
score = model.evaluate(x_test, y_test_1h, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.10849065432859202
Test accuracy: 0.95578825


<h3> Prediction Results </h3>

In [0]:
# Get predicted probability of being in spam or ham
results = model.predict_on_batch(x_test)

In [0]:
# Take the max to get the results
np.argmax(results, axis=1)

array([0, 0, 0, ..., 1, 1, 0])

<h3> Save Model </h3>

In [0]:
model.save_weights('email_classifier_weights.h5')

In [0]:
model.save('./email_classifier_model.h5')